In [1]:
%matplotlib inline

import datetime
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import numpy as np
import pandas as pd
import s3fs
import fsspec
import dask
from dask.distributed import performance_report, Client, progress

from eofs.xarray import Eof
import cartopy.crs as ccrs
import cartopy.feature as cfeature

/Users/ecwolff3/opt/anaconda3/envs/py3/lib/python3.10/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [12]:
# Start dask client
client = Client()
client

Task was destroyed but it is pending!
task: <Task pending name='Task-310166' coro=<ServerSession.with_document_locked() done, defined at /Users/ecwolff3/opt/anaconda3/envs/py3/lib/python3.10/site-packages/bokeh/server/session.py:78> wait_for=<Future finished result=<tornado.lock...x7fd5c9887130>> cb=[multi_future.<locals>.callback() at /Users/ecwolff3/opt/anaconda3/envs/py3/lib/python3.10/site-packages/tornado/gen.py:520]>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51778,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:51798,Total threads: 2
Dashboard: http://127.0.0.1:51800/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:51784,


# Problem 1

### Create SST Dataset

In [3]:
base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/'
base_url2 = '/e5.moda.an.sfc.128_034_sstk.ll025sc.'
base_url3 = '.nc'

# Set range of dates to retrieve data for
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

# Generate names for each file to be retrieved
file_list=[]
for dt in pr:
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # Create full file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    
    file_list.append(single_file)

In [16]:
# Access data from the THREDDS server
ds_SST = xr.open_mfdataset(file_list, parallel=True, engine='netcdf4').sel(
                        latitude=slice(65, -65, 4),
                        longitude=slice(120, 300, 4)).compute()

OSError: [Errno -90] NetCDF: file not found: b'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/2004/e5.moda.an.sfc.128_034_sstk.ll025sc.2004010100_2004120100.nc'

In [ ]:
# Create dataset grouped by month
gb_SST = ds_SST.SSTK.groupby('time.month')

In [ ]:
# Calculate SST anomlies by subtracting the monthly mean from each month
anom_SST = gb_SST - gb_SST.mean(dim='time')

### Create Precip Dataset

In [ ]:
base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.accumu/'
base_url2 = '/e5.moda.fc.sfc.accumu.128_228_tp.ll025sc.'
base_url3 = '.nc'

# Set range of dates to retrieve data for
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

# Generate names for each file to be retrieved
file_list=[]
for dt in pr:
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # Create full file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    
    file_list.append(single_file)

In [ ]:
# Access data from the THREDDS server
ds_precip = xr.open_mfdataset(file_list, parallel=True, engine='netcdf4').sel(
                        latitude=slice(65, -65, 1, 4),
                        longitude=slice(120, 300, 1, 4)).compute()

In [ ]:
# Create dataset grouped by month
gb_precip = ds_precip.TP.groupby('time.month')

In [ ]:
# Calculate precip anomlies by subtracting the monthly mean from each month
anom_precip = gb_precip - gb_precip.mean(dim='time')

#### Plot Example

In [ ]:
# Plot one month of SST anomalies
anom_SST.sel(time='1997-12-01', method='nearest').plot()

In [ ]:
# Plot one month of precip anomalies
anom_precip.sel(time='1997-12-01', method='nearest').plot()

# Problem 2

### Standardize SST

In [ ]:
# Calc the standard deviation of SSTs for each month
std_SST = ds_SST.groupby("time.month").std("time")

In [ ]:
# Standardize the anomalies by dividing by the standard deviation
standardized_SST = anom_SST/std_SST

In [ ]:
# Plot one month of standardized anomalies
standardized_SST['SSTK'].sel(time='1997-12-01').plot()

### Detrend SST

In [ ]:
# Define function to detrend data
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

# Run the detrend function
detrend_SST = detrend_dim(standardized_SST['SSTK'],'time',1)

#### Example Plot

In [ ]:
# Plot one month of detrended anomalies
detrend_SST.sel(time='1997-12-01', method='nearest').plot()

### Standardize Precip

In [ ]:
# Calc the standard deviation of precip for each month
std_precip = ds_precip.groupby("time.month").std("time")

In [ ]:
# Standardize the anomalies by dividing by the standard deviation
standardized_precip = anom_precip/std_precip

In [ ]:
# Plot one month of standardized anomalies
standardized_precip['TP'].sel(time='1997-12-01').plot()

### Detrend Precip

In [ ]:
# Define function to detrend data
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

# Run the detrend function
detrend_precip = detrend_dim(standardized_precip['TP'],'time',1)

#### Example Plot

In [ ]:
# Plot one month of detrended anomalies
detrend_precip.sel(time='1997-12-01', method='nearest').plot()

# Problem 3

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
coslat = np.cos(np.deg2rad(detrend_SST.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(detrend_SST, weights=wgts)

In [ ]:
eofs = solver.eofs(neofs=5)

In [ ]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
#fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2)
fig = plt.figure(figsize=(10, 10))
plt.suptitle('First 5 EOFs', fontsize=16)
ax1 = plt.subplot(3,2,1, projection=ccrs.PlateCarree(central_longitude=190))
fill1 = eofs[0].plot.contourf(ax=ax1, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill1, orientation='horizontal', shrink=0.8)
#cb.set_label('correlation coefficient', fontsize=12)
cb.ax.tick_params(labelsize=7)
ax1.set_title('EOF1', fontsize=16)

ax2 = plt.subplot(3,2,2, projection=ccrs.PlateCarree(central_longitude=190))
fill2 = eofs[1].plot.contourf(ax=ax2, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill2, orientation='horizontal', shrink=0.8)
#cb.set_label('correlation coefficient', fontsize=12)
cb.ax.tick_params(labelsize=7)
ax2.set_title('EOF2', fontsize=16)

ax3 = plt.subplot(3,2,3, projection=ccrs.PlateCarree(central_longitude=190))
fill3 = eofs[2].plot.contourf(ax=ax3, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill3, orientation='horizontal', shrink=0.8)
#cb.set_label('correlation coefficient', fontsize=12)
cb.ax.tick_params(labelsize=7)
ax3.set_title('EOF3', fontsize=16)

ax4 = plt.subplot(3,2,4, projection=ccrs.PlateCarree(central_longitude=190))
fill4 = eofs[3].plot.contourf(ax=ax4, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax4.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill4, orientation='horizontal', shrink=0.8)
#cb.set_label('correlation coefficient', fontsize=12)
cb.ax.tick_params(labelsize=7)
ax4.set_title('EOF4', fontsize=16)

ax5 = plt.subplot(3,2,5, projection=ccrs.PlateCarree(central_longitude=190))
fill5 = eofs[4].plot.contourf(ax=ax5, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax5.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill5, orientation='horizontal', shrink=0.8)
#cb.set_label('correlation coefficient', fontsize=12)
cb.ax.tick_params(labelsize=7)
ax5.set_title('EOF5', fontsize=16)

# Problem 4

In [ ]:
varfrac = solver.varianceFraction()

In [ ]:
# Plot the fraction of variance explained by each EOF
plt.figure(figsize=(11,6))
eof_num = range(1, 11)
plt.plot(eof_num, varfrac[0:10]*100, linewidth=2)
plt.plot(eof_num, varfrac[0:10]*100, linestyle='None', marker="o", color='r', 
            markersize=8)
plt.axhline(0, color='k')
plt.xticks(range(1, 11))
plt.title('Percent of the total variance represented by each EOF')
plt.xlabel('EOF #')
plt.ylabel('Variance Fraction')
plt.xlim(1, 10)
plt.ylim(np.min(varfrac)*100, (np.max(varfrac)*100)+0.01)

# Problem 5

In [ ]:
reconstruction = solver.reconstructedField(5)
reconstruction

In [ ]:
correlation = xr.corr(reconstruction, detrend_SST, dim='time')

In [ ]:
correlation.plot()

# Problem 6

In [ ]:
reconstruction_1 = solver.reconstructedField(1)

In [ ]:
precip_corr = xr.corr(reconstruction_1, detrend_precip, dim='time')

In [ ]:
precip_corr.plot()

In [ ]:
client.close()